# Empirical Analysis of Symmetric Nash Equilibrium in a Finite-Resource System

James Yu

11 December 2020

# Introduction

In Question 6 of Problem Set 4 of UBCV Econ 304, we explored a scenario where $n$ fisherman were fishing from a lake of infinite fish. Each fisherman had utility function:

$$
U(x_i, X) = x_i(b-c(X)), X = x_1+x_2+...+x_i+...+x_n
$$

which, because of the universal cost function, leads to a symmetric equilibrium.

The problem assumes there are infinite fish. However, in reality, we might have a lake with a finite number of fish. In this kind of scenario, we are now constrained such that:

$$
X \leq a \text{ (for some total fish level } a \text{)}
$$

(This assumes fishing all the fish in the lake is allowed. If we want to prevent this, we simply lower the value of $a$ such that it is less than the total number of fish).

For simplicity, we continue to assume everyone fishes at the same time. It is possible to extend the model to have a time-series component where the game is a sequential series of simultaneous games between fractions of the total number of fishermen, but that is beyond the scope of this notebook.

In this notebook, we will attempt to simulate such a setup using Python to see if a natural environment would converge to this equilibrium.

In general, this approaches the problem of a group of people or agents simultaneously vying to access a shared, limited resource. Other examples include eating a pizza at a party, or accessing compute resources on UBC's department servers, or firms purchasing advertisment space on the Times Square digital billboards.

In this notebook, we use the fishing example for consistency.

# Theory

In order to take a computational approach to the problem, we must define an explicit cost function. We define:

$$
c(X) = cX
$$

Given this, we have:

$$
U(x_i, X) = bx_i-cx_iX
$$

subject to $X \leq a$. Given this constraint, we set up a Lagrangian maximizer of:

$$
\mathcal{L}(x_i, X, \lambda) = \max\limits_{x_i} bx_i-cx_iX - \lambda(X - a)
$$

We, of course, have a total of $n$ of these problems to solve, one for each fisherman.
Taking the first-order condition, we get:

$$
\mathcal{L}_{x_i} = b-cX-cx_i - \lambda = 0
$$

Since we have a symmetric problem for each $i$, this states that:
$$
\forall i, j \in [0, n], b-cX-cx_i = b-cX-cx_j
$$

Cancelling common terms, we can simplify this to the following:

$$
\forall i, j \in [0, n], x_i = x_j
$$

Just like in the infinite case, we see each fisherman fishes the same amount of fish. Next, we see whether the total amount of fish fished is the total amount in the lake.

Suppose we didn't have a binding constraint; that is, $X < a$. We would then see $b-cX-cx_i = 0$ by complementary slackness.

Given that we have a symmetric Nash equilibrium, this simplifies to $b = c(x_i(n+1))$. Thus, $x_i = \frac{b}{c(n+1)}$.

Note that this is the solution from the infinite fish supply case. Compare this to the solution if the constraint binds, which is $\frac{a}{n}$. Which is better?

It depends on the parameters. Suppose $a = 10, b = 5, c = 0.5$. Then $\frac{10}{n} > \frac{10}{n+1}$, and it is optimal for the entire supply of fish to be fished.

In general, we must have $a(n+1) > \frac{b}{c}n$ for the boundary solution to be optimal.

For $n < a$, this will always be the case. However, if we had for example $a = 5, n = 6, \frac{b}{c} = 6$, we would get  $35 < 36$, making the other solution optimal.

Logically, if the number of fisherman is more than the number of fish, we would end up with a scenario where people get fractional numbers of fish, so we can ignore this case for the purposes of this notebook.

Thus, each fisherman will fish $\frac{a}{n}$ fish.

# Practical Result

Now, we can take the empirical approach to the Nash equilibrium. To do so, we will set up a Python program to simulate the actual result of $n$ "fishermen" attempting to access a limited resource of "fish". To represent the game, we have $n$ computational threads accessing a single pool variable. First, we import the threading library.

In [1]:
import threading

Next, we set up the model. We use a large list to represent the amount of fish each fisherman has caught, and a single variable representing the lake of fish. We also construct a function for each fisherman which "fishes", or deducts one from the pool variable and increments the fisherman variable in the list. When the pool has no fish left in it, the function stops. Also, let $a = 1000$ fish in the lake, and let $n = 100$ fisherman exist.

In [2]:
lake_cap = 1000 # capacity of the lake

lake = lake_cap

num_fishermen = 100

catches = [0]*num_fishermen # this duplicates the list num_fishermen times

def fish(index):
    global lake
    while lake > 0:
        lake -= 1
        catches[index] += 1

Next, we initialize a set of $n$ fisherman threads and let them run. After they finish, we print the result.

In [3]:
def program():
    global lake, lake_cap
    lake = lake_cap
    people = []

    for i in range(num_fishermen):
        catches[i] = 0
        person = threading.Thread(target = fish, args = (i,))
        people.append(person)
        person.start()

    for person in people:
        person.join()

    print(catches)
    
program()

[1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Each entry represents the number of fish that have been caught by the ith fisherman. Clearly, something is wrong, as the first fisherman has caught all the fish.

This is because the program is simulating a sequential game instead of a simultaneous game. Each fisherman is allowed to fish as many fish as they want at once, and because they're fast enough, they can catch the entire lake's worth before anyone else can. This is, of course, not what we want.

Instead, we need to modify the fishing function to yield to other fishermen after each fish is caught. This ensures a fisherman doesn't try to game the system and take the whole lake.

In [4]:
import time

def fish(index):
    global lake
    while lake > 0:
        lake -= 1
        catches[index] += 1
        time.sleep(0)

time.sleep(0) appears to do nothing, but this is really a method of "yielding" to other threads in the program. Now, we run the program again. Additionally, time.sleep(0) takes some number of CPU cycles to read, so it is not really a zero second delay.

In [5]:
program()

[25, 16, 24, 821, 28, 24, 9, 18, 20, 9, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Still not what we want. The fish are distributed over multiple fishermen, but severely non-uniformly.

Interestingly, if this code is run again, a different result appears each time.

In [6]:
program()
program()
program()

[87, 105, 95, 81, 60, 67, 100, 89, 43, 28, 46, 23, 20, 25, 27, 26, 20, 13, 20, 8, 5, 5, 2, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[27, 877, 18, 21, 24, 10, 6, 8, 6, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 997, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Let's try modifying the function to have a higher delay. Perhaps this will encourage sharing of the resource.

In [7]:
def fish(index):
    global lake
    while lake > 0:
        lake -= 1
        catches[index] += 1
        time.sleep(0.01)

program()

[12, 12, 14, 14, 12, 12, 14, 12, 12, 14, 14, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8]


This is significantly better but we still do not see the uniformity we expect. Note that what we are moving towards here is a sequential game where each fisherman takes one fish at a time, which isn't exactly the same as a simultaneous game where the fish are distributed and each fisherman takes some amount all at once, but approximates it closely because each first fisherman is not basing their actions on the actions of future fishermen.

We could modify the model so the first fisherman anticipates future ones, but then we'd just see the 1000 scenario from our first attempt.

Instead, we modify the time delay to be higher.

In [8]:
def fish(index):
    global lake
    while lake > 0:
        lake -= 1
        catches[index] += 1
        time.sleep(0.2)

program()

[11, 10, 11, 11, 10, 10, 11, 11, 10, 10, 10, 11, 11, 11, 11, 10, 11, 10, 11, 11, 10, 10, 10, 11, 10, 10, 10, 10, 11, 10, 11, 10, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 10, 10, 9, 10, 9, 10, 9, 9, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 9, 10, 10, 9, 10, 9, 10, 9, 9, 10, 10, 9, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 9]


This is just barely what we need. The only difference is that one fisherman consumes only 9 (instead of (1000/100 = 10)), and one, the first one, consumes 11. This does however seem to indicate that the game converges to the Nash equilibrium as the alotted delay between re-fishes increases. We try one more time increase.

In [9]:
def fish(index):
    global lake
    while lake > 0:
        lake -= 1
        catches[index] += 1
        time.sleep(0.3)

program()

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


This results in a perfect setup. Or at least it did when run once. It might be the case that it does not when run a certain number of times. We can however determine the smallest delay necessary to achieve the Nash equilibrium computationally by running the experiment again and again until we see it. To do so, we make the delay time a parameter of the fishing function.

In [10]:
def fish(index, wait):
    global lake
    while lake > 0:
        lake -= 1
        catches[index] += 1
        time.sleep(wait)

This requires us to redesign the program function slightly to account for the parameterized wait time, and also to not print the result.

In [11]:
def newprogram(wait):
    global lake, lake_cap
    lake = lake_cap
    people = []

    for i in range(num_fishermen):
        catches[i] = 0
        person = threading.Thread(target = fish, args = (i,wait,))
        people.append(person)
        person.start()

    for person in people:
        person.join()

    return catches

Finally, we run the program over a sample of times until we reach the result we need. We'll conduct a binary search over the times we found previously to determine the border time after which we get our equilibrium.

Theoretically, this would converge to a singular time. Once that happens, we print that time, as well as the result. We then print the time just before and after that result to see if this truly is the borderline.

A quick note: we assume that the Nash equilibrium is an integer number of fish here, which it is. If it turned out that the Nash equilibrium was fractional, the program would end up having an imbalanced final catch array. This would be slightly more difficult to look for and likely would not give the same result every time, so the integer values we receive from this setup benefits our experimentation. In practice, however, this may not happen.

In [ ]:
start = 0.2
end = 0.3

expected_catch = [lake_cap//num_fishermen]*num_fishermen

seen_midpoints = []

while True:
    midpoint = (start+end)/2
    seen_midpoints.append(midpoint)
    if newprogram(midpoint) == expected_catch:
        end = midpoint
    else:
        start = midpoint
    if start == end: break
        
print(seen_midpoints)
print("\n")
print(start, newprogram(start))
print("\n")
print(start-0.00001, newprogram(start-0.00001))
print("\n")
print(start+0.00001, newprogram(start+0.00001))
    

If this has not already executed, note it may take a while to compute due to the intensity of the search.

When run once, this search found the value to be around 0.2002 seconds of waiting, but this appears to vary. Another run shows 0.20002.

# Analysis

As the amount of time increases, the game starts to converge towards the setup where each fisherman simply takes one fish and then lets the next fisherman fish, as follows:

In [ ]:
lake = 1000

catch = [0]*100

cur_fisherman = 0
while lake > 0:
    lake -= 1
    catch[cur_fisherman] += 1
    cur_fisherman = (cur_fisherman + 1) % 100
    
print(catch)

Indeed, this results in the Nash equilibrium. This seems to imply the point at which the threaded model reaches the Nash equilibrium corresponds to the point where the amount of time a fisherman waits for all 99 other fishermen to finish fishing is equal to the waiting time. We test this theory below.

In [ ]:
def fish(index):
    global lake, start
    start = time.time()
    while lake > 0:
        lake -= 1
        catches[index] += 1
        if catches[index] == 2:
            print(f"I'm fisherman {index}. It took {time.time() - start} seconds to fish again.\n")
        print(f"I'm fisherman {index}. I have {catches[index]} fish and there are {lake} fish left.\n")
        time.sleep(start)

program()

In fact this theory was wrong. At first, indeed the wait time seems relatively close to the delay, and the fisherman all fish in order. However, as the game continues, the fishermen start fishing out of order. This means that, in fact, we are reaching the Nash equilibrium as a true convergence point, instead of just a convergence to the sequential take one and pass scenario. This means that given sufficient time to prevent fishermen from overfishing, the Nash equilibrium will be naturally reached. This also means that the simulation simulates rational players.

Again, it should be noted this game is not exactly identical to the proper simultaneous game. As only one fisherman fishes just one fish at a time, this is more of a restricted sequential game. However, because the players only optimize for themselves, and not with respect to anticipated future choices, we see the simutaneous equilibrium.

# Conclusion

In conclusion, the theoretical Nash equilibrium of a simultaneous finite-resource strategic game appears to be seen in empirical simulations. The equilibrium, an even distribution of resources over all the players, is a convergence point given players are gradually prevented from time-sensitive delays which would result in the possibility of one player taking everything in advance.

A more in-depth experiment would involve varying the $a$ and $n$ parameters to see if this result is observed for the general $a$ and $n$ case. However, this is beyond the scope of this notebook and is left as an exercise to the reader. Given that the behaviour of Python's threading system scales with the amount of threads, we expect to see the same result for larger values.